<a href="https://colab.research.google.com/github/abdulganiyMK007/freeCodeCamp-Project-Files/blob/master/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

## Setup input pipeline

In [ ]:
train_dataset = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
test_dataset = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])

In [ ]:
print(train_dataset.shape)
print(test_dataset.shape)

In [ ]:
train_dataset.head()

In [ ]:
train_dataset['type'] = train_dataset['type'].map({'ham': 0, 'spam': 1})
test_dataset['type'] = test_dataset['type'].map({'ham': 0, 'spam': 1})

In [ ]:
train_dataset.head()

In [ ]:
# train_input = train_dataset['msg']
# test_input = test_dataset['msg']

# train_labels = train_dataset.pop('type')
# test_labels = test_dataset.pop('type')

In [ ]:
BUFFER_SIZE = 250
BATCH_SIZE = 64

In [ ]:
# Convert the DataFrame to a tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(
    (
          tf.cast(train_dataset['msg'].values, tf.string),
          tf.cast(train_dataset['type'].values, tf.int32)
    )
)

In [ ]:
# Convert the DataFrame to a tf.data.Dataset
test_dataset = tf.data.Dataset.from_tensor_slices(
    (
          tf.cast(test_dataset['msg'].values, tf.string),
          tf.cast(test_dataset['type'].values, tf.int32)
    )
)

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for msg, label in train_dataset.take(1):
    print('texts: ', msg.numpy()[:3])
    print()
    print('labels: ', label.numpy()[:3])

## Create the text encoder

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

In [ ]:
encoded_example = encoder(msg)[:3].numpy()
encoded_example

In [ ]:
for n in range(3):
  print("Original: ", msg[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

## Create a Model

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')

# Convert sample_text to a tf.Tensor of strings
sample_text_tensor = tf.convert_to_tensor([sample_text])

# Predict using the model
predictions = model.predict(sample_text_tensor)

# predictions = model.predict(np.array([sample_text]))
print(predictions[0])

In [ ]:
# predict on a sample text with padding

padding = "the " * 2000

# Convert sample_text to a tf.Tensor of strings
sample_text_tensor = tf.convert_to_tensor([sample_text, padding])

# Predict using the model
predictions = model.predict(sample_text_tensor)

# predictions = model.predict(np.array([sample_text]))
print(predictions[0])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

## Train a Model

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

In [ ]:
model.summary()

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
text_input = ["Your input text here"]
text_input_tensor = tf.convert_to_tensor(text_input)
predictions = model.predict(text_input_tensor)

# The predictions will be between 0 and 1
print(predictions)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_text_tensor = tf.convert_to_tensor([pred_text])
    prediction = model.predict(pred_text_tensor)

    if prediction > 0.5:
        return [prediction, 'spam']
    else:
        return [prediction, 'ham']

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
